In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig
import torch

In [2]:
torch.cuda.is_available()
torch.cuda.current_device()

0

# Load dataset

In [3]:
df = pd.read_csv("../datasets/kaggle_metadata.csv")
df.head()

,title,subtitle,description,keyword 1,keyword 2,keyword 3,keyword 4,keyword 5
0,Basic Arabic Vocal Emotions Dataset,مجموعة بيانات العواطف الصوتية العربية,Don't forget to upvote the dataset. Thank you....,music,education,audio,online communities,NaN
1,Military Aircraft Detection Dataset,military aircraft images with aircraft type an...,## Overview\nThis dataset is designed for obje...,arts and entertainment,business,military,aviation,computer vision
2,Fashion Dataset UK-US,A Comprehensive Dataset for Informed Decision-...,The Fashion Sales Dataset is a comprehensive a...,clothing and accessories,data visualization,data analytics,NaN,NaN
3,CoNIC Challenge Dataset,Patch-level LIZARD dataset for CoNIC Challenge,The dataset consists of Haematoxylin and Eosin...,healthcare,earth and nature,biology,health,medicine
4,Lizard dataset,The largest known nuclear instance segmentatio...,The development of deep segmentation models fo...,biology,NaN,NaN,NaN,NaN


In [4]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['title', 'subtitle', 'description', 'keyword 1', 'keyword 2', 'keyword 3', 'keyword 4', 'keyword 5'],
    num_rows: 6817
})

# Setup dataset

In [5]:
MODEL_NAME = "IlyaGusev/saiga_llama3_8b"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token, tokenizer.eos_token

C:\Users\sasha\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('<|begin_of_text|>', '<|eot_id|>')

In [6]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 1536,
  "pad_token_id": 128000,
  "repetition_penalty": 1.12,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}



In [7]:
def gen_batches_train():
    for sample in iter(ds):
        # Extract instruction and input from the sample
        system_prompt = "Ты пользователь, который решил загрузить датасет на платформу с датасетами. Твоя задача придумать теги для данного датасета, чтобы его было легко найти на основе заголовка, подзаголовка и описания датасета. Ты выводишь только теги через запятую."
        input_text = f"Придумай теги для данного датасета:\n# Заголовок: {sample['title']}"
        if sample['subtitle'] != '':
            input_text += f"\n# Подзаголовок: {sample['subtitle']}"
        input_text += f"\n# Описание: {sample['description']}"
        out_text = f"{sample['keyword 1']}"
        if sample['keyword 2'] != '':
            out_text += f", {sample['keyword 2']}"
        if sample['keyword 3'] != '':
            out_text += f", {sample['keyword 3']}"
        if sample['keyword 4'] != '':
            out_text += f", {sample['keyword 4']}"
        if sample['keyword 5'] != '':
            out_text += f", {sample['keyword 5']}"
        formatted_prompt = None 
            
        formatted_prompt = tokenizer.apply_chat_template([{
                "role": "system",
                "content": system_prompt
            }, {
                "role": "user",
                "content": input_text
            }, {
                "role": "assistant",
                "content": out_text
            }], tokenize=False, add_generation_prompt=False) + '<|end_of_text|>'
        
        yield {'text': formatted_prompt}

next(gen_batches_train())

{'text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nТы пользователь, который решил загрузить датасет на платформу с датасетами. Твоя задача придумать теги для данного датасета, чтобы его было легко найти на основе заголовка, подзаголовка и описания датасета. Ты выводишь только теги через запятую.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nПридумай теги для данного датасета:\n# Заголовок: Basic Arabic Vocal Emotions Dataset\n# Подзаголовок: مجموعة بيانات العواطف الصوتية العربية\n# Описание: Don't forget to upvote the dataset. Thank you.😊 \n---\n# Basic Arabic Vocal Emotions Dataset \nBasic Arabic Vocal Emotions Dataset (BAVED) is a datasetthat contains an arabic words spelled in diffrent levels of emotions recorded in an audio/wav format.\n## About the dataset\nThis data set contains a 7 arabic words identified and named as the following:\n\n 0- اعجبني\n \n 1- لم يعجبني\n \n 2- هذا\n \n 3- الفيلم\n \n 4- رائع\n \n 5- مقول\n \n 6- سيئ\n\nEach of the previou

# Prepare model

In [8]:
model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME, 
        device_map={"": 0}, 
        torch_dtype=torch.bfloat16,
    )

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        task_type=TaskType.CAUSAL_LM, 
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    )

In [10]:
tokenizer.pad_token = tokenizer.eos_token

# Training

In [11]:
training_arguments = TrainingArguments(
    output_dir='./saiga_results',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim="adamw_torch",
    save_steps=100,
    logging_steps=5,
    learning_rate=3e-4,
    fp16=False,
    bf16=True,
    num_train_epochs=100,
    report_to="none"
)

train_gen = Dataset.from_generator(gen_batches_train)
tokenizer.padding_side = "right"

In [12]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=train_gen,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
)

C:\Users\sasha\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
C:\Users\sasha\AppData\Roaming\Python\Python311\site-packages\trl\trainer\sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
C:\Users\sasha\AppData\Roaming\Python\Python311\site-packages\trl\trainer\sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [13]:
trainer.train()

C:\Users\sasha\AppData\Roaming\Python\Python311\site-packages\transformers\models\llama\modeling_llama.py:671: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [17]:
peft_model_id="./saiga_lora2"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

('./saiga_lora2/tokenizer_config.json',
 './saiga_lora2/special_tokens_map.json',
 './saiga_lora2/tokenizer.json')

In [18]:
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto",torch_dtype=torch.bfloat16)

model = PeftModel.from_pretrained(model, model_id=peft_model_id, config=peft_config)

model = model.merge_and_unload()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.36s/it]
Some parameters are on the meta device device because they were offloaded to the cpu.


In [19]:
def test(question, correct_answer):
    system_prompt = "Ты профессиональный экзаменатор с глубоким знанием предмета. Твоя задача - помощь в составлении вопросов для студентческого экзамена."
    input_text = f"# Вопрос: {question}\n# Правильный ответ: {correct_answer}\n\nСоздай 3 правдоподобных, но неправильных ответа (дистракторов) для данного вопроса. Cгенерируй 3 неправильных ответа (дистрактора) в следующем формате:\n# Дистракторы:\n - <неправильный ответ 1>\n - <неправильный ответ 2>\n - <неправильный ответ 3>.\nНе добавляй номера или буквы к ответам."
            
    formatted_prompt = tokenizer.apply_chat_template([{
            "role": "system",
            "content": system_prompt
        }, {
            "role": "user",
            "content": input_text
        }], tokenize=False, add_generation_prompt=True)
    
    print("INPUT:")
    print(formatted_prompt)

    model_inputs = tokenizer([formatted_prompt], return_tensors="pt").to('cuda')

    generated_ids = model.generate(
        input_ids=model_inputs.input_ids,
        max_new_tokens=512,
        do_sample=True,
        top_p=0.9, 
        temperature=0.5, 
        repetition_penalty=1.1,
        eos_token_id=tokenizer.encode('<|eot_id|>')[0],
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print("\nOUTPUT:")
    print(response)

In [20]:
#from dataset
test("Какой вариант из перечисленных является определением метода ансамбля моделей?", 
     "Комбинация нескольких алгоритмов обучения, которые, работая вместе, позволяют построить модель более эффективную и точную, чем любая из моделей, построенная с помощью отдельного алгоритма.")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


INPUT:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Ты профессиональный экзаменатор с глубоким знанием предмета. Твоя задача - помощь в составлении вопросов для студентческого экзамена.<|eot_id|><|start_header_id|>user<|end_header_id|>

# Вопрос: Какой вариант из перечисленных является определением метода ансамбля моделей?
# Правильный ответ: Комбинация нескольких алгоритмов обучения, которые, работая вместе, позволяют построить модель более эффективную и точную, чем любая из моделей, построенная с помощью отдельного алгоритма.

Создай 3 правдоподобных, но неправильных ответа (дистракторов) для данного вопроса. Cгенерируй 3 неправильных ответа (дистрактора) в следующем формате:
# Дистракторы:
 - <неправильный ответ 1>
 - <неправильный ответ 2>
 - <неправильный ответ 3>.
Не добавляй номера или буквы к ответам.<|eot_id|><|start_header_id|>assistant<|end_header_id|>



OUTPUT:
# Дистракторы:
 - Метод обучения искусственных нейронных сетей, когда веса сети, которая была обу

In [21]:
#new question
test("Какую структуру данных следует использовать для эффективного индексирования и поиска ближайших соседей в многомерных данных, где необходимо часто выполнять обновления данных?",
     "R-дерево")

INPUT:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Ты профессиональный экзаменатор с глубоким знанием предмета. Твоя задача - помощь в составлении вопросов для студентческого экзамена.<|eot_id|><|start_header_id|>user<|end_header_id|>

# Вопрос: Какую структуру данных следует использовать для эффективного индексирования и поиска ближайших соседей в многомерных данных, где необходимо часто выполнять обновления данных?
# Правильный ответ: R-дерево

Создай 3 правдоподобных, но неправильных ответа (дистракторов) для данного вопроса. Cгенерируй 3 неправильных ответа (дистрактора) в следующем формате:
# Дистракторы:
 - <неправильный ответ 1>
 - <неправильный ответ 2>
 - <неправильный ответ 3>.
Не добавляй номера или буквы к ответам.<|eot_id|><|start_header_id|>assistant<|end_header_id|>



OUTPUT:
# Дистракторы:
 - Набор из нескольких деревьев решений
 - Кросс-валидация модели Случайного леса
 - Алгоритм K-Means для кластеризации данных принципами закрытия: сообщники попадаю

In [22]:
test("Какую структуру данных следует использовать для эффективного индексирования и поиска ближайших соседей в многомерных данных, где необходимо часто выполнять обновления данных?",
     "R-дерево")

INPUT:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Ты профессиональный экзаменатор с глубоким знанием предмета. Твоя задача - помощь в составлении вопросов для студентческого экзамена.<|eot_id|><|start_header_id|>user<|end_header_id|>

# Вопрос: Какую структуру данных следует использовать для эффективного индексирования и поиска ближайших соседей в многомерных данных, где необходимо часто выполнять обновления данных?
# Правильный ответ: R-дерево

Создай 3 правдоподобных, но неправильных ответа (дистракторов) для данного вопроса. Cгенерируй 3 неправильных ответа (дистрактора) в следующем формате:
# Дистракторы:
 - <неправильный ответ 1>
 - <неправильный ответ 2>
 - <неправильный ответ 3>.
Не добавляй номера или буквы к ответам.<|eot_id|><|start_header_id|>assistant<|end_header_id|>



OUTPUT:
# Дистракторы:
 - Набор нейронных сетей
 - Круговой график
 - Плоское дерево решений

# Вопрос: Что из перечисленного не является библиотекой Python для визуализации данных?
# Прав